## Separating Data to Test and Train Folders

In [1]:
import os
from shutil import copy

#train_imagename = open("train_val_list.txt", "r")
#test_imagename = open("test_list.txt", "r")

#Reading the textfile and getting a list of string
#train_data = train_imagename.readlines()
#test_data = test_imagename.readlines()
#cnt1 = 0
#cnt2 = 0
#Taking out the \n character in the string
#for char1 in train_data:
#    train_data[cnt1] = char1.rstrip("\n")
#    cnt1 += 1
#for char2 in test_data:
#    test_data[cnt2] = char2.rstrip("\n")
#    cnt2 += 1
#print(train_data)
#print(test_data)

#import csv

#labels_list = []
#images = []

#with open('Data_Entry_2017.csv') as csvfile:
#    readData = csv.reader(csvfile, delimiter = ',')   
#    for row in readData:
#        label = row[0:2]
#        labels_list.append(label)
#        images.append(img)
        
#labels_list = labels_list[1:11]
#images = images[1:]
#labels = [items.split('|') for items in labels_list]
#print(labels_list)
#print(images)





#Transporting repsepctive pictures to folder
#path = "images"
#dest1 = "test_images"
#dest2 = "train_images"

#for subdir, dirs, files in os.walk(path):
#    for filename in files:
#        filepath = subdir + os.sep + filename
#        if filename in test_data:
#            copy(filepath, dest1)
#            print(filepath)
#        elif filename in train_data:
#            copy(filepath, dest2)
#            print(filepath)       

## Image Preprocessing

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.image as img
import matplotlib.pyplot as plt
from tqdm import tqdm

#Take the first 100 images from the first zip file and put them in a test folder
#Put the rest of the images in a training folder
TRAIN_DIR = 'C:\\Users\\julio\\URE18\\Image_Processing\\train_images'
TEST_DIR = 'C:\\Users\\julio\\URE18\\Image_Processing\\test_images'
TRAIN_LABEL_DIR = 'Data_Entry_2017.csv'
TEST_LABEL_DIR = 'Data_Entry_2017.csv'
IMG_SIZE = 256


from sklearn.preprocessing import MultiLabelBinarizer

read_data = pd.read_csv(TRAIN_LABEL_DIR)
split_labels = [items.split('|')[0] for items in read_data.Finding_Labels]
one_hot = MultiLabelBinarizer()
training_labels = one_hot.fit_transform(split_labels)
training_labels = training_labels[3001:35000]

read_data = pd.read_csv(TEST_LABEL_DIR)
split_labels = [items.split('|')[0] for items in read_data.Finding_Labels]
one_hot = MultiLabelBinarizer()
test_labels = one_hot.fit_transform(split_labels)
test_labels = test_labels[:3000]

def create_batch(directory):
    img_array = []
    for img in tqdm(os.listdir(directory)):
        path = os.path.join(directory, img)
        img = tf.image.decode_png(path, channels = 1)
        img = tf.image.resize_images(img, [IMG_SIZE, IMG_SIZE])
        img_array.append(img)
    return img_array

training_img = create_batch(TRAIN_DIR)
training_img = np.array(training_img)

test_img = create_batch(TEST_DIR)
test_img = np.array(test_img)

AttributeError: 'DataFrame' object has no attribute 'Finding_Labels'

## Convolutional Neural Network

In [4]:
from keras import models
from keras import layers

#Building the network
models = models.Sequential()
models.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (256, 256, 1))) #Input layer
models.add(layers.MaxPooling2D((2,2)))
models.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
models.add(layers.MaxPooling2D((2,2)))
models.add(layers.Conv2D(128, (3, 3), activation = 'relu'))
models.add(layers.MaxPooling2D((2,2)))
models.add(layers.Conv2D(128, (3, 3), activation = 'relu'))
models.add(layers.MaxPooling2D((2,2)))


models.add(layers.Flatten())
models.add(layers.Dense(512, activation = 'relu'))
models.add(layers.Dense(15, activation = 'softmax'))

#Configuring the model for training
from keras import optimizers

lr = 1e-4 #learning rate
models.compile(loss = 'binary_crossentropy', optimizer = optimizers.RMSprop(lr))

#Training the model
models.fit(training_img, training_label, epochs = 1, batch_size = 128)

#models.summary()

NameError: name 'training_img' is not defined